In [1]:
# Imports

import pandas as pd                                         
import os                                       
from sklearn.feature_selection import SelectFromModel        
from sklearn.pipeline import Pipeline                      
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_predict, train_test_split
from sklearn.linear_model import LogisticRegression               
from sklearn.svm import SVC                       
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import ExtraTreeClassifier

Simplest possible implementation of NER

In [2]:
def read_data(input_path):
    train = pd.read_csv(input_path + "train_final_all.csv")
    test = pd.read_csv(input_path + "test_final_all.csv")
    return train, test

Prepare data

In [3]:
def data_prep(train, test, category):
    X_train = pd.concat([train["Token_index"], train.iloc[:,4:149]], axis = 1)
    y_train = train[category].values.tolist()

    X_test = pd.concat([test["Token_index"], test.iloc[:,4:149]], axis = 1)
    y_test = test[category].values.tolist()
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1000, test_size=0.3)

    return X_train, y_train, X_test, y_test

In [4]:
def train_model(clf, X_train, y_train):
    param_grid = [{}]
    grid_search = GridSearchCV(clf, param_grid,cv = StratifiedKFold(n_splits=3).
                               split(X_train, y_train), verbose=2)
    model = grid_search.fit(X_train, y_train)
    return model

In [5]:
def test_model(model, X_test, y_test):
    y_predictions = model.predict(X_test)
    report = classification_report(y_test, y_predictions)
    return report

In [17]:
input_path = 'D:/TU_Graz/Thesis/Datasets/Reddit_features/'
train, test = read_data(input_path) # Read data
clf_column = 'bio_movie' # Chose column to classify over
X_train, y_train, X_test, y_test = data_prep(train, test, clf_column) # Split the data to feats and target values (X and y) 

"""
Define the classifier. One of:

1. Naive Bayes
2. Support Vector Machines
3. Logistic Regression
4. Extra Tree
5. Decision Tree
6. Random Forest

"""
#clf = BernoulliNB()
#clf = SVC()
#clf = LogisticRegression(class_weight = 'balanced', penalty='l2', C=2, max_iter=10000)
#clf = ExtraTreeClassifier()
#clf = DecisionTreeClassifier(random_state=0)
clf = RandomForestClassifier(n_estimators=500, min_samples_split = 3)
#clf = CRF(algorithm='lbfgs', max_iterations=1000, all_possible_transitions=False)
#from sklearn.ensemble import AdaBoostClassifier
#clf = AdaBoostClassifier(n_estimators=100)

model = train_model(clf, X_train, y_train) # Train the model
report = test_model(model, X_test, y_test) # Test the model
print(report)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................................. , total= 6.6min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.6min remaining:    0.0s


[CV] ................................................. , total= 6.0min
[CV]  ................................................................
[CV] ................................................. , total= 5.9min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 18.5min finished


              precision    recall  f1-score   support

     B-movie       0.86      0.74      0.79      1287
     I-movie       0.85      0.70      0.76      1649
           O       0.97      0.99      0.98     22932

    accuracy                           0.96     25868
   macro avg       0.89      0.81      0.84     25868
weighted avg       0.95      0.96      0.96     25868

